# Dependencies

In [1]:
import tkinter as tk
from tkinter import *
from tkinter import HORIZONTAL,filedialog
from PIL import Image, ImageTk
import cv2
from skimage import io
from tensorflow.keras.models import load_model
import numpy as np
from skimage.transform import rescale, resize, downscale_local_mean
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
import os
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Input
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# GUI

In [3]:
window = tk.Tk()
window.title("Face Recognition Using SIAMESE NETWORKS")
canvas = tk.Canvas(window, width=1200, height=550,borderwidth=2).grid(columnspan=3, rowspan=30)

entry1 = tk.Button(window,width=30,height=1,text ='Capture Training Image', command = lambda:show_frames(),fg='black',bg='#fa9801',font='Helvetica 8 bold')
entry1.grid(column=0, row=0)

entry2 = tk.Button(window,width=30,height=1,text ='Save Training Image', command = lambda:open_file(),fg='black',bg='#fa9801',font='Helvetica 8 bold')
entry2.grid(column=0, row=1)

entry3 = tk.Button(window,width=30,height=1,text ='Capture to unlock', command = lambda:Test_img(),fg='black',bg='#fa9801',font='Helvetica 8 bold')
entry3.grid(column=0, row=2)

entry4 = tk.Button(window,width=30,height=1,text ='Select Training image', command = lambda:open_train(),fg='black',bg='#fa9801',font='Helvetica 8 bold')
entry4.grid(column=1, row=0)

entry5 = tk.Button(window,width=30,height=1,text ='Train Model', command = lambda:Train_model(),fg='black',bg='#fa9801',font='Helvetica 8 bold')
entry5.grid(column=1, row=1)

entry6 = tk.Button(window,width=30,height=1,text ='Select Test image', command = lambda:open_test(),fg='black',bg='#fa9801',font='Helvetica 8 bold')
entry6.grid(column=1, row=2)

label5 =Label(window,text="The distance2 is: ")
label5.grid(row=4, column=0)

label6 =Label(window,text = ".")
label6.grid(row=4, column=1)

label = Label(window)
label.grid(row=10, column=0)

label1 =Label(window)
label1.grid(row=10, column=1)

label7=Label(window,text="Phone Locked!!!!",bg='#f03e3e',font='Helvetica 20 bold')
label7.grid(row=50, column=1)   

SNM = load_model("C:\\Users\\raghavendra.s.k\\Desktop\\Files\\LV\\Siamese_model_VGG.h5")
Status = 'None'

#Capture Training image
def show_frames():
    global cv2image
    cap = cv2.VideoCapture(0)
    result = True
    while(result):
        cv2image= cv2.cvtColor(cap.read()[1],cv2.COLOR_BGR2RGB)
        result = False
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image = img)
    label.imgtk = imgtk
    label.configure(image=imgtk)
    entry1.configure(text="Training Image Captured",bg='#1fde72',font='Helvetica 8 bold')
    cap.release()
    cv2.destroyAllWindows()
    
#Model Training embedding
def open_file():
    global training_embedding,Status
    if Status == "None":
        training_embedding = embeddings(cv2image)
        Status = "Trained"
        entry2.configure(text="Model Trained",bg='#1fde72',font='Helvetica 8 bold')
    else:
        print("Model already trained, press reset")
    print(f"The current status is {Status}")       
    
#Capture Test Image    
def Test_img():
    cap = cv2.VideoCapture(0)
    result = True
    while(result):
        cv2image1= cv2.cvtColor(cap.read()[1],cv2.COLOR_BGR2RGB)
        result = False
    img = Image.fromarray(cv2image1)
    imgtk = ImageTk.PhotoImage(image = img)
    label1.imgtk = imgtk
    label1.configure(image=imgtk)
    Test_embed = embeddings(cv2image1)
    distance1 = euclidean_distance(training_embedding,Test_embed)
    label6.configure(text = distance1[0])      
    if distance1[0]<0.25:
        label7.configure(text='Phone Unlocked!!!',bg='#1fde72',font='Helvetica 20 bold')
    else:
        label7.configure(text='Phone Locked!!!',bg='#f03e3e',font='Helvetica 20 bold')
    
#Load train image file    
def open_train():
    global source_path
    source_path = filedialog.askopenfilename()
    if source_path :
        
#         cv2image2 = np.array(Image.open(source_path))
#         img = Image.fromarray(cv2image2)
#         imgtk = ImageTk.PhotoImage(image = img,size=(100,100))
#         label.imgtk = imgtk
#         label.configure(image=imgtk)
        
        entry4.configure(text = "Training file selected",bg='#1fde72',fg='black',font='Helvetica 8 bold')
        print(source_path)
    else:
        pass    

def Train_model():
    global training_embedding1
    if source_path :
        cv2image = resize(np.array(Image.open(source_path)),(224,224,3))
        print(cv2image.shape)
        training_embedding1 = embeddings(cv2image)
        Status = "Trained"
        entry2.configure(text="Model Trained",bg='#1fde72',font='Helvetica 8 bold') 
    else:
        entry4.configure(text = "Select File",bg='#f03e3e',fg='black',font='Helvetica 8 bold')
        
def open_test():
    global test_path,Test_embed1
    test_path = filedialog.askopenfilename()
    if test_path :
        
#         cv2image2 = np.array(Image.open(test_path))
#         img = Image.fromarray(cv2image2)
#         imgtk = ImageTk.PhotoImage(image = img,size=(100,100))
#         label1.imgtk = imgtk
#         label1.configure(image=imgtk)
       
        entry6.configure(text = "Test file selected",bg='#1fde72',fg='black',font='Helvetica 8 bold')    
        cv2image2 = resize(np.array(Image.open(test_path)),(224,224,3))
        Test_embed1 = embeddings(cv2image2)         
        distance1 = euclidean_distance(training_embedding1,Test_embed1)
        label6.configure(text = distance1[0]*100)
        if distance1[0]<0.002050:
            label7.configure(text='Phone Unlocked!!!',bg='#1fde72',font='Helvetica 20 bold')
        else:
            label7.configure(text='Phone Locked!!!',bg='#f03e3e',font='Helvetica 20 bold')
    else:
        pass           
        

def Image_resize(x):
    img = (x / 255.).astype(np.float32)
    img = cv2.resize(img, dsize = (224, 224))
    return img    

def embeddings(x):
    img = Image_resize(x)
    embedding_vector1 = SNM.predict(np.expand_dims(img, axis = 0))[0]
    return embedding_vector1

def euclidean_distance(x,y):
    a= K.sqrt(K.sum(K.square(x - y), axis=-1, keepdims=True))
    return a.numpy()


window.mainloop()

The current status is Trained


In [4]:
def euclidean_distance(x,y):
    '''Compute Euclidean Distance between two vectors'''
#     x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def Image_resize(x):
    img = io.imread(x)
    img= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = (img / 255.).astype(np.float32)
    img = cv2.resize(img, dsize = (224, 224))
    return img  

# def L2_distance(emb1, emb2):
#     return np.sum(np.square(emb1 - emb2))      

# def L2_distance2(emb1, emb2):
#     return np.sum(np.square(emb1)-np.square(emb2))


# def L2_distance1(emb1, emb2):
#     return 1-spatial.distance.cosine(emb1, emb2)

In [5]:
SNM = load_model("C:\\Users\\raghavendra.s.k\\Desktop\\Files\\LV\\Siamese_model_VGG.h5")
SNM1 = load_model("C:\\Users\\raghavendra.s.k\\Desktop\\Files\\LV\\Siamese_model_Mobilenet_V3_v1.h5")

In [6]:
image1 = SNM.predict(np.expand_dims(Image_resize("C:\\Users\\raghavendra.s.k\\Desktop\\Files\\LV\\Anchor\\Raghavendra.S.K.png"), axis = 0))
image2 = SNM.predict(np.expand_dims(Image_resize("C:\\Users\\raghavendra.s.k\\Desktop\\Files\\LV\\Negative\\Rajat.jpg"), axis = 0))
# C:\Users\raghavendra.s.k\Desktop\Files\LV\Negative\Rajat.jpg

In [7]:
n = euclidean_distance(image1,image2)

In [8]:
n.numpy()[0][0]

0.6613891

In [9]:
type(n)

tensorflow.python.framework.ops.EagerTensor

In [12]:
n.numpy()

array([[0.6613891]], dtype=float32)

In [9]:

root = Tk()
label=Label(root,text="I was Hidden")

def labelactive():
    label.pack()
    
def labeldeactive():
    label.pack_forget()
    
    
Button(root,text="Show",command=labelactive).pack()
Button(root,text="Hide",command=labeldeactive).pack()
root.mainloop()